Phase 1: Numerical Data Cleaning and Transformation
1. Data Import and Initial Inspection
The dataset was imported using Pandas and inspected for missing values, incorrect formats, and inconsistencies.
Key columns:
Star Ratings (star_rating)
Number of Ratings (num_ratings)
Number of Reviews (num_reviews)
Genres (genres)
Community Reviews (extracted into separate rating percentages)

In [ ]:
import pandas as pd

df = pd.read_csv("./Data/Goodreads-Books.csv")

In [ ]:
import ast # Convert JSON-like strings in 'community_reviews' to dictionary format
df['community_reviews'] = df['community_reviews'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract 1-star reviews count and percentage
df['1_star_reviews_num'] = df['community_reviews'].apply(lambda x: x['1_stars']['reviews_num'] if isinstance(x, dict) else 0)
df['1_star_reviews_percentage'] = df['community_reviews'].apply(lambda x: x['1_stars']['reviews_percentage'] if isinstance(x, dict) else 0)

# Convert genres from string to list
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Explode so each genre is a separate row
df_exploded = df.explode('genres')

In [ ]:
#extract 5 star reviews
df_exploded['5_star_reviews_num'] = df['community_reviews'].apply(
    lambda x: x['5_stars']['reviews_num'] if isinstance(x, dict) else 0)
df_exploded['5_star_reviews_percentage'] = df['community_reviews'].apply(
    lambda x: x['5_stars']['reviews_percentage'] if isinstance(x, dict) else 0)

In [ ]:
import numpy as np

In [ ]:
df_exploded['first_published'] = pd.to_datetime(df_exploded['first_published'], errors='coerce')
df_exploded['year_published'] = df_exploded['first_published'].dt.year  # Extract year only

In [ ]:
df_exploded['num_reviews'] = df_exploded['num_reviews'].fillna(0).astype(int)
df_exploded['num_ratings'] = df_exploded['num_ratings'].fillna(0).astype(int)

In [ ]:
#exploding community review column
df_exploded['2_star_reviews_num'] = df_exploded['community_reviews'].apply(
    lambda x: x['2_stars']['reviews_num'] if isinstance(x, dict) and '2_stars' in x else 0)
df_exploded['2_star_reviews_percentage'] = df_exploded['community_reviews'].apply(
    lambda x: x['2_stars']['reviews_percentage'] if isinstance(x, dict) and '2_stars' in x else 0)

df_exploded['3_star_reviews_num'] = df_exploded['community_reviews'].apply(
    lambda x: x['3_stars']['reviews_num'] if isinstance(x, dict) and '3_stars' in x else 0)
df_exploded['3_star_reviews_percentage'] = df_exploded['community_reviews'].apply(
    lambda x: x['3_stars']['reviews_percentage'] if isinstance(x, dict) and '3_stars' in x else 0)

df_exploded['4_star_reviews_num'] = df_exploded['community_reviews'].apply(
    lambda x: x['4_stars']['reviews_num'] if isinstance(x, dict) and '4_stars' in x else 0)
df_exploded['4_star_reviews_percentage'] = df_exploded['community_reviews'].apply(
    lambda x: x['4_stars']['reviews_percentage'] if isinstance(x, dict) and '4_stars' in x else 0)

In [ ]:
# Drop columns
df_exploded = df_exploded.drop(columns=['community_reviews'])

df_exploded = df_exploded.drop(columns=['url'])

df_exploded = df_exploded.drop(columns=['id'])

df_exploded = df_exploded.drop(columns=['first_published'])

In [ ]:
#cleaning author column just in case I want to use it later
df_exploded['author'] = df_exploded['author'].apply(lambda x: x.replace('[', '').replace(']', '').replace('"', '').replace("'", "").strip())

In [ ]:
# Remove rows where star_rating, num_ratings, and num_reviews are 0 and genres is NaN
df_exploded = df_exploded[~((df_exploded['star_rating'] == 0) & 
          (df_exploded['num_ratings'] == 0) & 
          (df_exploded['num_reviews'] == 0) & 
          (df_exploded['genres'].isna()))]

In [ ]:
df_exploded['genres'].nunique()

In [ ]:
mask = (
    (df_exploded['star_rating'].fillna(0) == 0) &
    (df_exploded['num_ratings'].fillna(0) == 0) &
    (df_exploded['num_reviews'].fillna(0) == 0) &
    (
        df_exploded['genres'].isna() | 
        (df_exploded['genres'].str.lower() == 'nan') | 
        (df_exploded['genres'].str.strip() == '') ))

In [ ]:
df_masked = df_exploded[mask]
print(df_masked)

In [ ]:
# View unique values in specific columns
unique_genres = df_masked['genres'].unique()
unique_star_rating = df_masked['star_rating'].unique()
unique_num_ratings = df_masked['num_ratings'].unique()
unique_num_reviews = df_masked['num_reviews'].unique()

print("Unique values in genres:", unique_genres)
print("Unique values in star_rating:", unique_star_rating)
print("Unique values in num_ratings:", unique_num_ratings)
print("Unique values in num_reviews:", unique_num_reviews)

In [ ]:
df_exploded = df_exploded[~mask]

In [ ]:
#before
print("before shape:", df.shape)

#after
print("after shape:", df_exploded.shape)

In [ ]:
df_exploded = df_exploded.drop(columns=['kindle_price'])

In [ ]:
df_exploded.sample(30)

In [ ]:
print(df_exploded['star_rating'].value_counts())

In [ ]:
df_exploded.to_csv("cleaned_numerical_data_only.csv", index=True)

Phase 2: Numerical Data Exploration and Visualization
1. Top 20 Most Common Genres
Objective: Identify the most frequent book genres.

In [ ]:
# visualizations

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Exclude rows where 'genres' is 'nan'
filtered_genres = df_exploded.loc[df_exploded['genres'].str.lower() != 'nan', 'genres']

# Now get the top 20 genres
genre_counts = filtered_genres.value_counts().head(20)

plt.figure(figsize=(12, 6))
sns.barplot(x=genre_counts.values, y=genre_counts.index, palette="viridis")
plt.xlabel("Number of Books")
plt.ylabel("Genre")
plt.title("Top 20 Most Common Genres in the Dataset (excluding 'nan')")
plt.show()

### 2. **Genres with the Highest Percentage of 1-Star Reviews**
**Objective**: Identify which genres tend to receive the most negative ratings.

In [ ]:
# Filter out rows where 'genres' is NaN or 'nan'
genre_1star_reviews = filtered_genres.groupby('genres')['1_star_reviews_percentage'].mean() \
                                     .sort_values(ascending=False).head(20)

plt.figure(figsize=(12, 6))
sns.barplot(x=genre_1star_reviews.values, y=genre_1star_reviews.index, palette="magma")
plt.xlabel("Average Percentage of 1-Star Reviews")
plt.ylabel("Genre")
plt.title("Top 20 Genres with Highest Proportion of 1-Star Reviews (excluding NaN)")
plt.show()

### 3. **Relationship Between Number of Ratings and Star Rating**
**Objective**: Understand if books with more ratings tend to have higher or lower average ratings.

In [ ]:
# Drop rows with NaN in 'star_rating' or 'num_ratings'
ratings_df = df_exploded.dropna(subset=['star_rating', 'num_ratings'])

plt.figure(figsize=(10, 6))
sns.scatterplot(x=ratings_df['num_ratings'], y=ratings_df['star_rating'], alpha=0.5)
plt.xscale("log")  # Log scale to handle large numbers
plt.xlabel("Number of Ratings (log scale)")
plt.ylabel("Average Star Rating")
plt.title("Relationship Between Number of Ratings and Star Rating")
plt.show()

### 4. **Distribution of Star Ratings**
**Objective**: Visualize how ratings are spread across all books.

In [ ]:
# Drop NaN values in 'star_rating'
star_rating_series = df_exploded['star_rating'].dropna()

plt.figure(figsize=(8, 5))
sns.histplot(star_rating_series, bins=20, kde=True, color="blue")
plt.xlabel("Star Rating")
plt.ylabel("Number of Books")
plt.title("Distribution of Star Ratings (excluding NaN)")
plt.show()

### 5. **Correlation Matrix**
**Objective**: Identify numerical relationships between rating factors.

In [ ]:
# Select only the numerical columns
num_cols = ['star_rating', 'num_ratings', 'num_reviews',
            '1_star_reviews_percentage', '5_star_reviews_percentage',
            '2_star_reviews_percentage', '3_star_reviews_percentage',
            '4_star_reviews_percentage', 
            '1_star_reviews_num', '5_star_reviews_num', 
            '2_star_reviews_num', '3_star_reviews_num', '4_star_reviews_num', 
            'year_published']

# Calculate correlation matrix
corr_matrix = df_exploded[num_cols].corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix of Numerical Variables")
plt.show()